# Anomalieerkennung mit Autoencodern
**Jan-Philipp Schulze, Fraunhofer AISEC, 2020**

In diesem Tutorial wollen wir Anomalien mit einem speziellen Neuronalen Netz (NN), dem Autoencoder, erkennen.
Autoencoder lernen, die Eingabe zu komprimieren und am Ende zu rekonstruieren.
Trainiert man sie nur auf normalen Daten, so haben sie Schwierigkeiten, anomale Daten wiederherzustellen - dies kann man messen und zur Anomalieerkennung benutzen.

Währenddessen lernen wir die gängige Machine Learning (ML) Pipeline kennen.
Diese besteht aus:
* **Daten einlesen** (read): Kennenlernen der Daten, z.B. was ist enthalten, wie sind sie verteilt?
* **Daten vorverarbeiten** (preprocess): Aufbereitung der Daten, sodass der ML Algorithmus sie verarbeiten kann.
* **ML Modell trainieren** (train): Das ML Modell "lernt" aus den Daten.
* **ML Modell nutzen** (predict): Vorhersage treffen.
* **Leistung evaluieren** (evaluate): Performance des ML Modells abschätzen.


## Daten einlesen
Um die Ergebnisse zu visualisieren, benutzen wir kein klassisches Anomalieerkennungsdatensatz, sondern einen bekannten Datensatz aus der Bildverarbeitung: Fashion-MNIST.
Dieser ist bereits bei TF2 mit dabei.

Wie in ML üblich bezeichnet ``x`` die Eingabe (hier: das Bild) und ``y`` die Ausgabe (hier: eine Klasse).
Wir unterscheiden zwischen Daten auf denen wir trainieren und auf denen wir testen.
In echten ML Szenarios fernab dieses Tutorials sollten wir darüberhinaus einen Datensatz zum Validieren erstellen.

In [ ]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

### Aufgaben
1. Lernt die Daten kennen:
    1. Wie viele Samples haben wir? Was sind ihre Dimensionen?
    2. Welche Werte haben die Daten? Warum?
2. Filtert nach Daten aus der Klasse 0. Was zeigen sie?

In [ ]:
# Euer Code

## Daten vorverarbeiten
NN lernen über Gradienten.
Diese haben bessere Eigenschaften, wenn die Daten skaliert bzw. normalisiert werden.
Ohne Preprocessing kann es sein, dass das NN nicht oder nur sehr langsam lernt.

### Aufgaben
1. Skaliere die Daten, sodass sie zwischen 0 und 1 liegen.
2. Unser simpler Autoencoder akzeptiert nur 1-dimensionale Eingaben. Formatiere entsprechend.
3. Für unsere Anomalieerkennung nehmen wir alle Bilder aus Klasse 0 als normal an, alle aus 1 als anomal.
    1. Reduziere den Trainingsdatensatz auf die Klasse 0.
    2. Reduziere den Testdatensatz auf Klasse 0 und 1.

In [ ]:
# Euer Code

## ML Modell trainieren
Während des Trainings wird das ML Modell auf die Trainingsdaten angepasst:
In unserem Fall lernt der Autoencoder, die Trainingsdaten zu rekonstruieren.

### Deep Learning mit TensorFlow2
Wir bauen unseren Autoencoder in TensorFlow2 (TF2) auf.
TF2 kennt drei Arten, ein NN aufzubauen:
* Sequentiell
* Funktional
* Subklassen

Wir werden eine Mischung aus Sequenzen und Subklassen verwenden.

In [ ]:
class Autoencoder(tf.keras.Model):
    def __init__(self):
        super(Autoencoder, self).__init__()

        self.m_encoder = None
        self.m_decoder = None

    def build(self, input_shape):
        # Construct the encoder
        m_encoder = tf.keras.Sequential(name="encoder")
        m_encoder.add(tf.keras.layers.Dense(800, activation="relu", input_shape=input_shape))
        m_encoder.add(tf.keras.layers.Dense(400, activation="relu"))
        m_encoder.add(tf.keras.layers.Dense(100, activation="relu"))

        self.m_encoder = m_encoder

        # Construct the decoder
        m_decoder = tf.keras.Sequential(name="decoder")
        m_decoder.add(tf.keras.layers.Dense(400, activation="relu", input_shape=m_encoder.output_shape[1:]))
        m_decoder.add(tf.keras.layers.Dense(800, activation="relu"))
        m_decoder.add(tf.keras.layers.Dense(input_shape[-1], activation="sigmoid"))

        self.m_decoder = m_decoder

    def call(self, input_data, training=None, mask=None):
        # Concatenate our autoencoder
        t_code = self.m_encoder(input_data, training=training, mask=mask)
        t_out = self.m_decoder(t_code, training=training, mask=mask)

        return t_out

Bevor wir den Autoencoder trainieren können, müssen wir noch angeben, welches Ziel das Modell hat:
Dazu definiert man eine Fehlerfunktion (Loss-Function).
Beim Autoencoder wollen wir die Eingabe möglichst genau rekonstruiert haben.
Subklassen von ``tf.keras.Model`` haben darüberhinaus eine ``fit``-Funktion, die zum Trainieren benutzt wird.

In [ ]:
# Create an autoencoder
autoencoder = Autoencoder()
# Define the loss
autoencoder.compile(loss="binary_crossentropy", optimizer="adam")
# Train
autoencoder.fit(x=x_train, y=x_train, epochs=10, batch_size=128, verbose=2)

## ML Modell nutzen
Wir haben nun einen Autoencoder, der gut mit den Trainingsdaten umgehen kann.
Wie sieht es jedoch auf den bisher unbekannten Testdaten aus?

### Aufgaben
1. Benutze die ``predict``-Funktion, um:
    1. Ein Sample der normalen Klasse 0 zu rekonstruieren,
    2. Ein Sample der anomalen Klasse 1 zu rekonstruieren.
2. Plotte die Ergebnisse.

In [ ]:
# Euer Code

### Anomalien finden
Wie wir gesehen haben, hat der Autoencoder Probleme, anomale Eingaben zu rekonstruieren.
Dies können wir zur Anomalieerkennung nutzen: Wir messen den Abstand zwischen der Eingabe und der Ausgabe.
Für anomale Eingaben sollte dieser höher liegen.

In diesem Falle sagen wir: Alles, was weit weg vom normalen Rekonstruktionsfehler liegt, ist anomal.
Wir modellieren dies mithilfe einer Gaussverteilung.
Beachtet: Wir müssten dies eigentlich auf dem Validierungsdatensatz machen.

In [ ]:
# Normal reconstruction error
re_normal = np.mean(np.square(x_norm - pred_norm), axis=1)
# Anomaly threshold based on a Gaussian
re_normal_mean = np.mean(re_normal)
re_normal_std = np.std(re_normal)
anom_thresh = re_normal_mean + re_normal_std

# Reconstruction error of unknown samples
re_unknown = np.mean(np.square(x_test - pred_test), axis=1)
y_unknown = np.zeros_like(y_test)
y_unknown[re_unknown > anom_thresh] = 1


## Leistung evaluieren
Aus dem vorherigen Schritt haben wir eine Reihe an Vorhersagen: 0 für normal, 1 für anomal.
Aber haben wir eine gute Anomalieerkennung gebaut?
Dies evaluieren wir durch Metriken.
In der Vorlesung habt ihr Precision and Recall kennengelernt.

In [ ]:
from sklearn.metrics import precision_score, recall_score
# Compute metrics
pred_precision = precision_score(y_true=y_test, y_pred=y_unknown)
pred_recall = recall_score(y_true=y_test, y_pred=y_unknown)

print(f"Precision: {pred_precision:.3f}")
print(f"Recall: {pred_recall:.3f}")
